In [ ]:
#
# Prototyping for the following ... now it's about optimization
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from shapely import Polygon
from   udist_scatterplots_via_sectors          import UDistScatterPlotsViaSectors
from   udist_scatterplots_via_sectors_tile_opt import UDistScatterPlotsViaSectorsTileOpt
import time
import rtsvg
rt = rtsvg.RACETrack()
import random
num_of_pts   = [100, 200, 50]
#num_of_pts   = [2_000, 4_000, 1_000]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])

_iterations_ = 2

t1 = time.time()
udspvs          = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_, debug=True)
t2 = time.time()
udspvs_tile_opt = UDistScatterPlotsViaSectorsTileOpt(_xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_, debug=True, num_of_tiles=64)
t3 = time.time()
# 351 Points | 512 Iterations | 5.66s Polars Time | 10.68s Polars Tile Opt Time | (M1 Pro 16G)
# 7001 Points | 2 Iterations | 12.30s Polars Time | 4.97s Polars Tile Opt Time (32 tiles) | (M1 Pro 16G)  # After uniquifying the xoyo sectors
# 7001 Points | 2 Iterations | 9.38s Polars Time | 2.95s Polars Tile Opt Time (64 tiles) | (M1 Pro 16G) 
print(f'{len(_xvals_)} Points | {_iterations_} Iterations | {t2-t1:.2f}s Polars Time | {t3-t2:.2f}s Polars Tile Opt Time | (M1 Pro 16G) ')
#rt.tile([udspvs.animateIterations(animation_dur="4s"), udspvs_tile_opt.animateIterations(animation_dur="4s")], spacer=10)

In [ ]:
self = udspvs_tile_opt
w_step, h_step = 384, 384
pt_i, iter = 15, 0
_tiles_ = []

_df_ = self.df_tile_determinations[iter]
svg = [f'<svg x="0" y="0" width="{w_step}" height="{h_step}" viewBox="0.0 0.0 1.0 1.0" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="0" y="0" width="1.0" height="1.0" x="0" y="0" fill="#ffffff" />')
_xiyis_ = set()
for i in range(len(_df_)):
    xi, yi      = _df_['xi'][i], _df_['yi'][i]
    _xiyis_.add((xi,yi))
for _xy_ in _xiyis_:
    xi, yi = _xy_
    x0,y0,x1,y1 = self.tileBounds(xi,yi)
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="none" stroke="#000000" stroke-width="0.001"/>')
for i in range(len(_df_)):
    x, y, w, c = _df_['x'][i], _df_['y'][i], _df_['w'][i], _df_['c'][i]
    svg.append(f'<circle cx="{x}" cy="{y}" r="0.003" fill="{c}" />')
svg.append(f'<text x="1.0" y="0.06" text-anchor="end" font-size="0.05" fill="#000000">{len(_xiyis_)} Tiles</text>')
_df_ = _df_.filter(pl.col('__index__') == pt_i)
svg.append('</svg>')

_tiles_.append(''.join(svg))

In [ ]:
_df_ = self.df_tile_sums[iter]
svg = [f'<svg x="0" y="0" width="{w_step}" height="{h_step}" viewBox="0.0 0.0 1.0 1.0" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="0" y="0" width="1.0" height="1.0" x="0" y="0" fill="#ffffff" />')
_max_ = _df_['tile_sum'].max()
for i in range(len(_df_)):
    xi, yi, tile_sum = _df_['xi'][i], _df_['yi'][i], _df_['tile_sum'][i]
    x0,y0,x1,y1 = self.tileBounds(xi,yi)
    _v_     = int(255 - 255 * tile_sum / _max_)
    _color_ = f'#{_v_:02x}{_v_:02x}{_v_:02x}'
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="{_color_}" stroke="none"/>')
svg.append(f'<text x="1.0" y="0.06" text-anchor="end" font-size="0.05" fill="#000000">{len(_df_)} Tiles</text>')
svg.append('</svg>')
_tiles_.append(''.join(svg))


In [ ]:
_df_ = self.df_cross_join_tile_offsets[iter].filter(pl.col('__index__') == pt_i)

svg = [f'<svg x="0" y="0" width="{w_step}" height="{h_step}" viewBox="0.0 0.0 1.0 1.0" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="0" y="0" width="1.0" height="1.0" x="0" y="0" fill="#ffffff" />')
for i in range(len(_df_)):
    xi, yi      = _df_['xi'][i] + _df_['xo'][i],  _df_['yi'][i] + _df_['yo'][i]
    x0,y0,x1,y1 = self.tileBounds(xi,yi)
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="none" stroke="#000000" stroke-width="0.002"/>')
svg.append(f'<text x="1.0" y="0.06" text-anchor="end" font-size="0.05" fill="#000000">{len(_df_)} Tiles</text>')
svg.append('</svg>')

_tiles_.append(''.join(svg))

In [ ]:
_df_ = self.df_join_sector_info[iter].filter(pl.col('__index__') == pt_i)

svg = [f'<svg x="0" y="0" width="{w_step}" height="{h_step}" viewBox="0.0 0.0 1.0 1.0" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="0" y="0" width="1.0" height="1.0" x="0" y="0" fill="#ffffff" />')
for xi in range(self.num_of_tiles):
    for yi in range(self.num_of_tiles):
        x0,y0,x1,y1 = self.tileBounds(xi,yi)
        svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="none" stroke="#000000" stroke-width="0.0002"/>')
seen_already = set()
for i in range(len(_df_)):
    xi, yi, sector = _df_['xi_tile_sums'][i],  _df_['yi_tile_sums'][i], _df_['sector'][i]
    x0,y0,x1,y1    = self.tileBounds(xi,yi)
    _color_        = rt.co_mgr.getColor(sector)
    if sector == -1: _color_ = '#a0a0a0'
    if (xi,yi) in seen_already: _stroke_ = '#ff0000'
    else:                       _stroke_ = 'none'
    seen_already.add((xi,yi))
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="{_color_}" stroke="{_stroke_}" stroke-width="0.005" />')
x, y = _df_['x'][0], _df_['y'][0]
svg.append(f'<circle cx="{x}" cy="{y}" r="0.01" fill="#000000" stroke="none"/>')
for i in range(len(self.df_sector_angles[iter])):
    u, v = self.df_sector_angles[iter]['a0u'][i], self.df_sector_angles[iter]['a0v'][i]
    svg.append(f'<line x1="{x}" y1="{y}" x2="{x+2*u}" y2="{y+2*v}" stroke="#000000" stroke-width="0.003"/>')

svg.append(f'<text x="1.0" y="0.06" text-anchor="end" font-size="0.05" fill="#000000">{len(_df_)} Tiles</text>')
svg.append('</svg>')

_tiles_.append(''.join(svg))

In [ ]:
_df_ = self.df_separate_easy_way[iter].filter(pl.col('__index__') == pt_i)
svg = [f'<svg x="0" y="0" width="{w_step}" height="{h_step}" viewBox="0.0 0.0 1.0 1.0" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="0" y="0" width="1.0" height="1.0" x="0" y="0" fill="#ffffff" />')
for xi in range(self.num_of_tiles):
    for yi in range(self.num_of_tiles):
        x0,y0,x1,y1 = self.tileBounds(xi,yi)
        svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="none" stroke="#000000" stroke-width="0.0002"/>')
seen_already = set()
for i in range(len(_df_)):
    xi, yi, sector = _df_['xi_tile_sums'][i],  _df_['yi_tile_sums'][i], _df_['sector'][i]
    x0,y0,x1,y1    = self.tileBounds(xi,yi)
    _color_        = rt.co_mgr.getColor(sector)
    if sector == -1: _color_ = '#a0a0a0'
    if (xi,yi) in seen_already: _stroke_ = '#ff0000'
    else:                       _stroke_ = 'none'
    seen_already.add((xi,yi))
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="{_color_}" stroke="{_stroke_}" stroke-width="0.005" />')
x, y = _df_['x'][0], _df_['y'][0]
svg.append(f'<circle cx="{x}" cy="{y}" r="0.01" fill="#000000" stroke="none"/>')
for i in range(len(self.df_sector_angles[iter])):
    u, v = self.df_sector_angles[iter]['a0u'][i], self.df_sector_angles[iter]['a0v'][i]
    svg.append(f'<line x1="{x}" y1="{y}" x2="{x+2*u}" y2="{y+2*v}" stroke="#000000" stroke-width="0.003"/>')

svg.append(f'<text x="1.0" y="0.06" text-anchor="end" font-size="0.05" fill="#000000">{len(_df_)} Tiles / Easy</text>')
svg.append('</svg>')

_tiles_.append(''.join(svg))

In [ ]:
_df_ = self.df_separate_hard_way[iter].filter(pl.col('__index__') == pt_i)
svg = [f'<svg x="0" y="0" width="{w_step}" height="{h_step}" viewBox="0.0 0.0 1.0 1.0" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="0" y="0" width="1.0" height="1.0" x="0" y="0" fill="#ffffff" />')
for xi in range(self.num_of_tiles):
    for yi in range(self.num_of_tiles):
        x0,y0,x1,y1 = self.tileBounds(xi,yi)
        svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="none" stroke="#000000" stroke-width="0.0002"/>')
for i in range(len(_df_)):
    xi, yi, sector = _df_['xi_tile_sums'][i],  _df_['yi_tile_sums'][i], _df_['sector'][i]
    x0,y0,x1,y1    = self.tileBounds(xi,yi)
    _color_        = rt.co_mgr.getColor(sector)
    if sector == -1: _color_ = '#a0a0a0'
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="{_color_}" stroke="{_stroke_}" stroke-width="0.005" />')
x, y = _df_['x'][0], _df_['y'][0]
svg.append(f'<circle cx="{x}" cy="{y}" r="0.01" fill="#000000" stroke="none"/>')
for i in range(len(self.df_sector_angles[iter])):
    u, v = self.df_sector_angles[iter]['a0u'][i], self.df_sector_angles[iter]['a0v'][i]
    svg.append(f'<line x1="{x}" y1="{y}" x2="{x+2*u}" y2="{y+2*v}" stroke="#000000" stroke-width="0.002"/>')
_tile_count_ = 0
for k, k_df in _df_.group_by(['xi_tile_sums', 'yi_tile_sums']): _tile_count_ += 1
svg.append(f'<text x="1.0" y="0.06" text-anchor="end" font-size="0.05" fill="#000000">{len(_df_)} Points / Hard</text>')
svg.append(f'<text x="1.0" y="0.12" text-anchor="end" font-size="0.05" fill="#000000">{_tile_count_} Tiles / Hard</text>')
svg.append('</svg>')

_tiles_.append(''.join(svg))
rt.table(_tiles_, spacer=10, per_row=4)

In [ ]:
_pts_ = 10_000
_lu_  = {'x0':[], 'y0':[], 'x1':[], 'y1':[], 'x':[], 'y':[]}
_x0_, _y0_, _x1_, _y1_ = random.random(), random.random(), random.random(), random.random()
for i in range(_pts_):
    _lu_['x0'].append(_x0_), _lu_['y0'].append(_y0_)
    _lu_['x1'].append(_x1_), _lu_['y1'].append(_y1_)
    _lu_['x'] .append(random.random()), _lu_['y'] .append(random.random())
_df_ = pl.DataFrame(_lu_)

t0 = time.time()
_dx_, _dy_ = pl.col('x') - pl.col('x0'), pl.col('y') - pl.col('y0')
_df_       = _df_.with_columns(((16*(pl.arctan2(_dy_, _dx_) + pl.lit(pi))/(pl.lit(2*pi))).cast(pl.Int64)).alias('sector'))
t1 = time.time()

print(f'{t1-t0:.3f}s arctan2')

t0 = time.time()
_df_ = _df_.with_columns(
            pl.when((pl.col('x0') - pl.col('x')) * (pl.col('y1') - pl.col('y')) - (pl.col('y0') - pl.col('y')) * (pl.col('x1') - pl.col('x')) > 0.0)
            .then(1)
            .otherwise(0)
            .alias('right')
)
t1 = time.time()

print(f'{t1-t0:.3f}s pointRightOfLine')

# 10M points / arctan2          0.394s / M1 Pro (16G)
# 10M points / pointRightOfLine 0.058s / M1 Pro (16G) # Almost 7x faster ...
rt.xy(_df_, x_field='x', y_field='y', color_by='right', dot_size='small')

In [ ]:
_df_ = self.df_xoyo_sector
xo_min, yo_min, xo_max, yo_max = _df_['xo'].min(), _df_['yo'].min(), _df_['xo'].max(), _df_['yo'].max()
dx, dy = xo_max - xo_min, yo_max - yo_min
svg = [f'<svg x="0" y="0" width="900" height="900" viewBox="{xo_min-1} {yo_min-1} {dx+3} {dy+3}" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="{xo_min-1}" y="{yo_min-1}" width="{dx+3}" height="{dy+3}" fill="#ffffff" />')
for i in range(len(_df_)):
    xo, yo, sector, u, v, rsector, lsector = _df_['xo'][i], _df_['yo'][i], _df_['sector'][i], _df_['u'][i], _df_['v'][i], _df_['rsector'][i], _df_['lsector'][i]
    if sector != -1: 
        _color_ = rt.co_mgr.getColor(sector)
        svg.append(f'<rect x="{xo}" y="{yo}" width="1" height="1" fill="{_color_}" stroke="none" />')
    elif rsector is not None:
        _color_ = '#a0a0a0'
        svg.append(f'<rect x="{xo}" y="{yo}" width="1" height="1" fill="{_color_}" stroke="none" />')
    
svg.append('</svg>')
rt.tile([''.join(svg)])